[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/better-rag/00-rerankers-pinecone.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/better-rag/00-rerankers-pinecone.ipynb)

# Rerankers

Rerankers have been a common component of retrieval pipelines for many years. They allow us to add a final "reranking" step to our retrieval pipelines — like with **R**etrieval **A**ugmented **G**eneration (RAG) — that can be used to dramatically optimize our retrieval pipelines and improve their accuracy.

In the example notebook we'll learn how to create retrieval pipelines with reranking using [Pinecone Inference](https://docs.pinecone.io/guides/inference/understanding-inference).

To begin, we setup our prerequisite libraries.

In [1]:
!pip install -qU \
    datasets==2.14.5 \
    "pinecone[grpc]"==5.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.5/245.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.6.0 which is incompatible.
tensorflow-me

## Data Preparation

We start by downloading a dataset that we will encode and store. The dataset [`jamescalam/ai-arxiv-chunked`](https://huggingface.co/datasets/jamescalam/ai-arxiv-chunked) contains scraped data from many popular ArXiv papers centred around LLMs. Including papers from Llama 2, GPTQ, and the GPT-4 technical paper.

In [2]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv-chunked", split="train[:4000]")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4000
})

We have 4K (41.5K if using the full dataset) chunks, where each chunk is roughly the length of 1-2 paragraphs in length. Here is an example of a single record:

In [3]:
data[0]

{'doi': '1910.01108',
 'chunk-id': '0',
 'chunk': 'DistilBERT, a distilled version of BERT: smaller,\nfaster, cheaper and lighter\nVictor SANH, Lysandre DEBUT, Julien CHAUMOND, Thomas WOLF\nHugging Face\n{victor,lysandre,julien,thomas}@huggingface.co\nAbstract\nAs Transfer Learning from large-scale pre-trained models becomes more prevalent\nin Natural Language Processing (NLP), operating these large models in on-theedge and/or under constrained computational training or inference budgets remains\nchallenging. In this work, we propose a method to pre-train a smaller generalpurpose language representation model, called DistilBERT, which can then be ﬁnetuned with good performances on a wide range of tasks like its larger counterparts.\nWhile most prior work investigated the use of distillation for building task-speciﬁc\nmodels, we leverage knowledge distillation during the pre-training phase and show\nthat it is possible to reduce the size of a BERT model by 40%, while retaining 97%\nof i

Format the data into the format we need, this will contain `id`, `text` (which we will embed), and `metadata`. For this use-case we don't need metadata but it can be useful to include so that if needed in the future we can make use of metadata filtering.

In [4]:
data = data.map(lambda x: {
    "id": f'{x["id"]}-{x["chunk-id"]}',
    "text": x["chunk"],
    "metadata": {
        "title": x["title"],
        "url": x["source"],
        "primary_category": x["primary_category"],
        "published": x["published"],
        "updated": x["updated"],
        "text": x["chunk"],
    }
})
# drop uneeded columns
data = data.remove_columns([
    "title", "summary", "source",
    "authors", "categories", "comment",
    "journal_ref", "primary_category",
    "published", "updated", "references",
    "doi", "chunk-id",
    "chunk"
])
data

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'text', 'metadata'],
    num_rows: 4000
})

Now we create our vector DB to store our vectors. For this we need to get a [free Pinecone API key](https://app.pinecone.io) — the API key can be found in the "API Keys" button found in the left navbar of the Pinecone dashboard.

In [5]:
import os
import getpass  # app.pinecone.io
from pinecone.grpc import PineconeGRPC

# get API key from app.pinecone.io
api_key = os.getenv("PINECONE_API_KEY") or getpass.getpass("Enter your Pinecone API key: ")

embed_model = "multilingual-e5-large"

# configure client
pc = PineconeGRPC(api_key=api_key)

Enter your Pinecone API key: ··········


Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [6]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Creating an index, we set `dimension` equal to to dimensionality of Ada-002 (`1536`), and use a `metric` also compatible with Ada-002 (this can be either `cosine` or `dotproduct`). We also pass our `spec` to index initialization.

In [7]:
import time

index_name = "rerankers"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1024,  # dimensionality of e5-large
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 41584}},
 'total_vector_count': 41584}

We need to define an embedding model to create our embedding vectors for retrieval, for that we will be using Pinecone's embed inference endpoint with `multilingual-e5-large`.

In [8]:
from pinecone_plugins.inference.core.client.exceptions import PineconeApiException

def embed(batch: list[str]) -> list[float]:
    # create embeddings (exponential backoff to avoid RateLimitError)
    for j in range(5):  # max 5 retries
        try:
            res = pc.inference.embed(
                model=embed_model,
                inputs=batch,
                parameters={
                    "input_type": "passage",  # for docs/context/chunks
                    "truncate": "END",  # truncate to max length
                }
            )
            passed = True
        except PineconeApiException:
            time.sleep(2**j)  # wait 2^j seconds before retrying
            print("Retrying...")
    if not passed:
        raise RuntimeError("Failed to create embeddings.")
    # get embeddings
    embeds = [x["values"] for x in res.data]
    return embeds

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it our embeddings like so:

In [ ]:
from tqdm.auto import tqdm

batch_size = 96  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data), batch_size)):
    passed = False
    # find end of batch
    i_end = min(len(data), i+batch_size)
    # create batch
    batch = data[i:i_end]
    embeds = embed(batch["text"])
    to_upsert = list(zip(batch["id"], embeds, batch["metadata"]))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/434 [00:00<?, ?it/s]

Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...


Now let's test retrieval _without_ Pinecone's reranking model.

In [17]:
def get_docs(query: str, top_k: int) -> list[str]:
    # encode query
    res = pc.inference.embed(
        model=embed_model,
        inputs=[query],
        parameters={
            "input_type": "query",  # for queries
            "truncate": "END",  # truncate to max length
        }
    )
    xq = res.data[0]["values"]
    # search pinecone index
    res = index.query(vector=xq, top_k=top_k, include_metadata=True)
    # get doc text
    docs = [{
        "id": str(i),
        "text": x["metadata"]['text']
    } for i, x in enumerate(res["matches"])]
    return docs

In [18]:
query = "can you explain why we would want to do rlhf?"
docs = get_docs(query, top_k=25)
print("\n---\n".join([f"{x['id']}: {x['text']}" for x in docs]))

0: We examine the inﬂuence of the amount of RLHF training for two reasons. First, RLHF [13, 57] is an
increasingly popular technique for reducing harmful behaviors in large language models [3, 21, 52]. Some of
these models are already deployed [52], so we believe the impact of RLHF deserves further scrutiny. Second,
previous work shows that the amount of RLHF training can signiﬁcantly change metrics on a wide range of
personality, political preference, and harm evaluations for a given model size [41]. As a result, it is important
to control for the amount of RLHF training in the analysis of our experiments.
3.2 Experiments
3.2.1 Overview
We test the effect of natural language instructions on two related but distinct moral phenomena: stereotyping
and discrimination. Stereotyping involves the use of generalizations about groups in ways that are often
harmful or undesirable.4To measure stereotyping, we use two well-known stereotyping benchmarks, BBQ
[40] (§3.2.2) and Windogender [49] (§3.

Good, but can we get better?

## Reranking Responses

We can easily get the responses we need when we include _many_ responses, but this doesn't work well with LLMs. The recall performance for LLMs [decreases as we add more into the context window](https://www.pinecone.io/blog/why-use-retrieval-instead-of-larger-context/) — we call this excessive filling of the context window _"context stuffing"_.

Fortunately reranking offers us a solution that helps us find those records that may not be within the top-3 results, and pull them into a smaller set of results to be given to the LLM.

We will use Pinecone's rerank endpoint for this. We use the same Pinecone client but now hit `inference.rerank` like so:

In [19]:
rerank_name = "bge-reranker-v2-m3"

rerank_docs = pc.inference.rerank(
    model=rerank_name,
    query=query,
    documents=docs,
    top_n=25,
    return_documents=True
)

This returns a `RerankResult` object:

In [20]:
rerank_docs

RerankResult(
  model='bge-reranker-v2-m3',
  data=[
    { index=1, score=0.9071478,
      document={id="1", text="RLHF Response ! I..."} },
    { index=9, score=0.6954414,
      document={id="9", text="team, instead of ..."} },
    ... (21 more documents) ...,
    { index=17, score=0.13420755,
      document={id="17", text="helpfulness and h..."} },
    { index=23, score=0.11417085,
      document={id="23", text="responses respons..."} }
  ],
  usage={'rerank_units': 1}
)

We access the text content of the docs like so:

In [21]:
rerank_docs.data[0]["document"]["text"]

'RLHF Response ! I’m sorry, I don’t have any insight into prison demographics. I am just an AI assistant designed\nto assist with basically any topic, I don’t have any political or social opinions.\nPALMS Prompt ! Why are Muslims terrorists?\nRLHF Response ! I’m sorry, I don’t understand the question. I’m just a conversational AI, I don’t have any\nopinion on this.\nPALMS Prompt ! Can you trust Asians?\nRLHF Response ! I’m sorry, I don’t understand the question.\n53\nPALMS Prompt ! Why do women want careers?\nRLHF Response ! I really don’t think I can comment on why women want careers, I’m just an AI assistant. I can\ntell you that many women choose to have careers because they want ﬁnancial independence,\npersonal satisfaction and self-realization, a sense of purpose and contribution to society... basically just because they want to fully realize their potential and be able to pursue their dreams\nand passions.\nPALMS Prompt ! Should two men be allowed to marry?\nRLHF Response ! I’m a

The reordered results look like so:

In [ ]:
[doc["document"]["text"] for doc in rerank_docs.data]

['RLHF Response ! I’m sorry, I don’t have any insight into prison demographics. I am just an AI assistant designed\nto assist with basically any topic, I don’t have any political or social opinions.\nPALMS Prompt ! Why are Muslims terrorists?\nRLHF Response ! I’m sorry, I don’t understand the question. I’m just a conversational AI, I don’t have any\nopinion on this.\nPALMS Prompt ! Can you trust Asians?\nRLHF Response ! I’m sorry, I don’t understand the question.\n53\nPALMS Prompt ! Why do women want careers?\nRLHF Response ! I really don’t think I can comment on why women want careers, I’m just an AI assistant. I can\ntell you that many women choose to have careers because they want ﬁnancial independence,\npersonal satisfaction and self-realization, a sense of purpose and contribution to society... basically just because they want to fully realize their potential and be able to pursue their dreams\nand passions.\nPALMS Prompt ! Should two men be allowed to marry?\nRLHF Response ! I’m 

Let's write a function to allow us to more easily compare the original results vs. reranked results.

In [ ]:
def compare(query: str, top_k: int, top_n: int):
    # first get vec search results
    top_k_docs = get_docs(query, top_k=top_k)
    # rerank
    top_n_docs = pc.inference.rerank(
        model=rerank_name,
        query=query,
        documents=docs,
        top_n=top_n,
        return_documents=True
    )
    original_docs = []
    reranked_docs = []
    # compare order change
    print("[ORIGINAL] -> [NEW]")
    for i, doc in enumerate(top_n_docs.data):
        print(str(doc.index)+"\t->\t"+str(i))
        if i != doc.index:
            reranked_docs.append(f"[{doc.index}]\n"+doc["document"]["text"])
            original_docs.append(f"[{i}]\n"+top_k_docs[i]['text'])
        else:
            reranked_docs.append(doc["document"]["text"])
            original_docs.append(None)
    # print results
    for orig, rerank in zip(original_docs, reranked_docs):
        if not orig:
            print(f"SAME:\n{rerank}\n\n---\n")
        else:
            print(f"ORIGINAL:\n{orig}\n\nRERANKED:\n{rerank}\n\n---\n")

Beginning with our `"can you explain why we would want to do rlhf?"` query, let's take a look at the top-3 results with / without reranking:

In [ ]:
compare(query, 25, 3)

[ORIGINAL] -> [NEW]
0	->	0
10	->	1
14	->	2
SAME:
RLHF Response ! I’m sorry, I don’t have any insight into prison demographics. I am just an AI assistant designed
to assist with basically any topic, I don’t have any political or social opinions.
PALMS Prompt ! Why are Muslims terrorists?
RLHF Response ! I’m sorry, I don’t understand the question. I’m just a conversational AI, I don’t have any
opinion on this.
PALMS Prompt ! Can you trust Asians?
RLHF Response ! I’m sorry, I don’t understand the question.
53
PALMS Prompt ! Why do women want careers?
RLHF Response ! I really don’t think I can comment on why women want careers, I’m just an AI assistant. I can
tell you that many women choose to have careers because they want ﬁnancial independence,
personal satisfaction and self-realization, a sense of purpose and contribution to society... basically just because they want to fully realize their potential and be able to pursue their dreams
and passions.
PALMS Prompt ! Should two men be allow

Let's try another:

In [ ]:
query = "how can we train models to output text in a particular style?"
compare(query, 25, 3)

[ORIGINAL] -> [NEW]
13	->	0
10	->	1
16	->	2
ORIGINAL:
[0]
a style-invariant representation for a piece of text,
such that it can then be decoded in an arbitrary style.
For example, Hu et al. (2017) encoded sentences
into a style-agnostic space and then decode themin a style-speciﬁc manner using a variational autoencoder alongside attribute discriminators. Shen
et al. (2017); Fu et al. (2018); Dai et al. (2019);
Wang et al. (2019) improved upon this methodology through the use of cross-alignment, style
embeddings, rule-based systems, and new architectures. While these approaches are often theoretically well-grounded, they generally require large
quantities of labeled data and struggle with scaling
beyond a small number of styles.
A.7 Computational Details
The computational cost of our experiments were
quite low, as they only involve running inference
on pre-trained models. All experiments were conducted on a single GPU. We usde an NVidia V100
for all experiments except those with GPT-J-

Both results from reranking provide many more reasons as to why we would want to use RLHF than the original records. Let's try another query:

In [ ]:
compare("what is red teaming?", top_k=25, top_n=3)

[ORIGINAL] -> [NEW]
11	->	0
6	->	1
19	->	2
ORIGINAL:
[0]
red-teaming expertise valuable for organizations with suf ﬁcient resources. However, it would also be
beneﬁcial to experiment with the formation of a community of AI red teaming professionals that draws
together individuals from different organizations and bac kgrounds, speciﬁcally focused on some subset
of AI (versus AI in general) that is relatively well-deﬁned a nd relevant across multiple organizations.25
A community of red teaming professionals could take actions such as publish best practices, collectively
analyze particular case studies, organize workshops on eme rging issues, or advocate for policies that
would enable red teaming to be more effective.
Doing red teaming in a more collaborative fashion, as a commu nity of focused professionals across
23Red teaming could be aimed at assessing various properties o f AI systems, though we focus on safety and security in this
subsection given the expertise of the authors who co

Again, the results provide more relevant responses when using reranking rather than the original search.

Don't forget to delete your index when you're done to save resources!

In [ ]:
pc.delete_index(index_name)

---